In [2]:
import requests
from bs4 import BeautifulSoup
import time
import random
import string
import numpy as np
import pandas as pd
import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [1]:
!pip install selenium

In [3]:
def google_search(x,n):
    driver = webdriver.Chrome('chromedriver')
    driver.get(f'https://www.google.com/search?q={x}')
    next_page_times = n
    http=[]
    for _page in range(next_page_times):
    
        soup = BeautifulSoup(driver.page_source,'html.parser')
        items=soup.find_all('div', attrs={'class':'yuRUbf'})

        for i in items:
            # 網址
            h=i.find('a')['href']
            if 'cdc.gov.tw' in h :
                pass
            else:
                http.append(h)
        # Wait
        time.sleep(random.uniform(1, 5))

        # Turn to the next page
        try:
            driver.find_element_by_link_text('下一頁').click()
        except:
            print('Search Early Stopping.')
            driver.close()
            exit()
    driver.close()
    return http

In [4]:
def count_word(x):
    chinese=[]
    english=[]
    number=[]
    en=''
    num=''
    for i in list(x):

        if i in string.ascii_letters:
            en+=i
        elif i.isdigit():
            num+=i
        else:
            if en !='':
                english.append(en)
            if num!='':
                number.append(num)
            en=''
            num=''
            if i !=' ':
                chinese.append(i)
    words=len(english)+len(chinese)+len(number)
    return words

In [5]:
def liberal(x):
    ltn_news=BeautifulSoup(x.text, 'html.parser')
    #標題
    try:
        t=ltn_news.find('title').text
        t2=t.split('-')
        titles=t2[0]
    except:
        titles='錯誤'
    #分類
    try:
        sention=ltn_news.find('script',type="application/ld+json").contents[0].split('\n')[7].split(':')[1][2:-2]
    except:
        sention='錯誤'
    #日期
    try:
        date=ltn_news.find('meta',property="article:published_time")['content'][0:10]
    except:
        date='錯誤'
    #內文
    try:
        con_list=ltn_news.find_all('p')[3:]

        content=''
        for i  in con_list:
            try:
                if i.text[-1] in ['。','？','?','！'] and i.text!='「武漢肺炎專區」請點此，更多相關訊息，帶您第一手掌握。':
                    content+=i.text
                else:
                    pass
            except:
                pass
    except:
        content='錯誤'
    #字數
    try:
        words=count_word(content)
    except:
        words='錯誤'
    #作者
    try:
        a1=content.find('〔')
        a2=content.find('／')
        author=content[a1+1:a2]
    except:
        author='錯誤'
    if '記者' in author :
        author=author.replace('記者','')
    #圖片數
    try:
        n_p=len(ltn_news.find_all('div',attrs={'class':'photo boxTitle'}))+len(ltn_news.find_all('span',attrs={'class':'ph_b ph_d1'}))
    except:
        n_p='錯誤'
    #影片數
    try:
        v=ltn_news.find_all('iframe')
        v2=[]
        v3=[]
        for i in v:
            try:
                if 'autoplay' in i['allow']:
                    if 'facebook' in i['src']:
                        v3.append(i)
                    else:
                        v2.append(i)
            except:
                pass
        v_p=len(v2)
        if len(v3)!=0:
            all_p='出現fb網址'
        elif v_p !=0:
            all_p=f'{n_p}（+{v_p}影片）'
        else:
            all_p=n_p
    except:
        all_p=0
    if '疫苗' in content :
        return ['','',author,'',sention,date,titles,content,'','','','','',words,'',all_p,'']
    else:
        return ['','','','','','','','','','','','','','','','','']

In [6]:
def web_crawl(x,n):
    http=google_search(x,n)
    ltn=[]
    for i in range(0,len(http)):
        ltn.append(requests.get(http[i]))
        #每爬一個休息1-5秒(random抽)，怕被抓
        time.sleep(random.uniform(1, 5))
    liberal_news_result=list(map(liberal,ltn))
    liberal_news_df=pd.DataFrame(liberal_news_result)
    liberal_news_df=liberal_news_df.rename(index=str,columns={0:'',1:'',2:'作者',3:'',4:'記者路線',5:'日期',6:'標題',7:'內文',8:'',9:'',10:'',11:'',12:'',13:'字數',14:'',15:"附圖（幀）",16:''})
    liberal_news_df['網址']=http
    liberal_news_df=liberal_news_df[liberal_news_df.標題!='']
    return liberal_news_df

In [7]:
df=web_crawl('疫苗 after:2021-03-11 before:2021-03-21 site:ltn.com.tw',1)

In [10]:
df['網址'][0]

'https://health.ltn.com.tw/article/breakingnews/3468418'

In [12]:
df.to_excel('自由時報實驗.xlsx')

# 下面是測試區

In [29]:
r=[]
for i in range(0,len(web_list)):
    r1 = requests.get(web_list[i])
    r.append(r1)
    time.sleep(random.uniform(1, 5))
http=[]
title=[]
for i in range(0,len(r)):
    if r[0].status_code == requests.codes.ok:
      # 以 BeautifulSoup 解析 HTML 原始碼
        soup = BeautifulSoup(r[0].text, 'html.parser')

      #觀察 HTML 原始碼
      # print(soup.prettify())

      # 以 CSS 的選擇器來抓取 Google 的搜尋結果
        items = soup.select('a[href^="/url"]')

        items= items[:-2]
        for i in items:
        # 網址
            http.append(i.get('href')[7:].split('&')[0])

In [33]:
http

['https://news.ltn.com.tw/news/world/breakingnews/3399330',
 'https://news.ltn.com.tw/news/world/breakingnews/3399648',
 'https://news.ltn.com.tw/news/world/breakingnews/3398730',
 'https://ec.ltn.com.tw/article/paper/1422893',
 'https://news.ltn.com.tw/news/politics/breakingnews/3399358',
 'https://ec.ltn.com.tw/article/breakingnews/3398879',
 'https://ec.ltn.com.tw/article/breakingnews/3398629',
 'https://news.ltn.com.tw/news/world/breakingnews/3398538',
 'https://news.ltn.com.tw/news/politics/breakingnews/3399568',
 'https://news.ltn.com.tw/news/world/breakingnews/3399099',
 'https://news.ltn.com.tw/news/world/breakingnews/3399330',
 'https://news.ltn.com.tw/news/world/breakingnews/3399648',
 'https://news.ltn.com.tw/news/world/breakingnews/3398730',
 'https://ec.ltn.com.tw/article/paper/1422893',
 'https://news.ltn.com.tw/news/politics/breakingnews/3399358',
 'https://ec.ltn.com.tw/article/breakingnews/3398879',
 'https://ec.ltn.com.tw/article/breakingnews/3398629',
 'https://news.

In [298]:
def liberal(x):
    ltn_news=BeautifulSoup(x.text, 'html.parser')
    #標題
    t=ltn_news.find('title').text
    t2=t.split('-')
    title=t2[0]
    #分類
    sention=t2[1]
    #日期
    date=ltn_news.find('meta',property="article:published_time")['content'][0:10]
    #內文
    con_list=ltn_news.find_all('p')[3:]

    content=''
    for i  in con_list:
        try:
            if i.text[-1] in ['。','？','?','！'] and i.text!='「武漢肺炎專區」請點此，更多相關訊息，帶您第一手掌握。':
                content+=i.text
            else:
                pass
        except:
            pass
    #字數
    words=count_word(content)
    #作者
    a1=content.find('〔')
    a2=content.find('／')
    author=content[a1+1:a2]
    if '記者' in author :
        author=author.replace('記者','')
    n_p=len(ltn_news.find_all('div',attrs={'class':'photo boxTitle'}))
    v_p=len(ltn_news.find_all('iframe',allow='accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture'))
    if v_p !=0:
        all_p=f'{n_p}（+{v_p}影片）'
    else:
        all_p=n_p
    if '疫苗' in content :
        return [title,content,words,sention,date,author,all_p]
    elif words==0 :
        return [title,content,words,sention,date,author,all_p]
    else:
        return['','','','','','','']

In [299]:
liberal(ltn[3])

['英國允許「混合接種」不同藥廠疫苗 挨轟放棄科學 ',
 '〔即時新聞／綜合報導〕英國受到武漢肺炎（新型冠狀病毒病，COVID-19）變種病毒肆虐，疫情持續擴散，英國國內已批准兩款疫苗以應對大規模接種計畫，根據英國當局公布的最新疫苗施打指南，將允許疫苗「混合接種」，特定的情況下2劑可分別施打不同藥廠疫苗。消息一出也引發專家批評，認為當局已「放棄科學」。綜合外媒報導，英國國內目前已開始疫苗施打計畫，提供民眾藥廠輝瑞（Pfizer）、BioNTech疫苗，以及藥廠阿斯特捷利康（AstraZeneca）、牛津大學的疫苗。在英國批准牛津疫苗後，當局也更新接種指南，若是無法獲得與第1劑相同疫苗，或是不清楚第1劑疫苗製造商的情況下，可施打不同藥廠疫苗。但指南中也呼籲盡量接種兩劑相同的疫苗，強調只有在民眾處於「高風險」或被認定是「不太可能再次接種疫苗」等緊急狀況下，才建議民眾施打不同藥廠疫苗。據報導，輝瑞疫苗與牛津疫苗具相似性，英國也將針對混合疫苗進行臨床試驗。這項新的規定不同於美國疫苗接種指南，根據美國疾病管制與預防中心（CDC）指示，武漢肺炎疫苗「不可互換」，尚未評估的混合疫苗的安全性和有效性，強調接種的兩劑疫苗應使用相同藥廠產品。儘管引發批評，仍有英國政府專家認為，混合疫苗有可能提升民眾對武漢肺炎病毒的防禦力。英國疫苗工作小組負責人賓厄姆（Kate Bingham）則提到，英國政府希望能讓疫苗效用最大化，加強免疫力以保護民眾健康。部分科學家認為，英國新的接種指南是一場「賭注」，康乃爾大學（Cornell University）疫苗專家摩爾（John Moore）表示，沒有任何數據證明這個想法可行，怒批英國官員似乎已「完全放棄科學」，只是透過瞎猜試圖找到混亂中的出路。',
 648,
 ' 國際 ',
 '2021-01-02',
 '即時新聞',
 1]

In [12]:
ltn=[]
for i in range(0,len(http)):
    ltn.append(requests.get(http[i]))
    time.sleep(random.uniform(1, 5))

In [175]:
ltn_news=BeautifulSoup(requests.get('https://news.ltn.com.tw/news/life/breakingnews/3103655').text, 'html.parser')

In [12]:
ltn_news=BeautifulSoup(ltn[3].text, 'html.parser')

NameError: name 'ltn' is not defined

In [126]:
print(ltn_news.prettify())

<!DOCTYPE html>
<html lang="zh-Hant-TW">
 <head>
  <script>
   var pvDomain = 'pv.ltn.com.tw';
  </script>
  <script>
   var pvServer = 'pv.ltn.com.tw';
  </script>
  <script>
   var interstitialSlot;
  </script>
  <script>
   var staticSlot = false;
  </script>
  <script src="https://cache.ltn.com.tw/js/c.js">
  </script>
  <base href="https://news.ltn.com.tw/"/>
  <meta content="R" name="ltn:device"/>
  <title>
   多管道確保疫苗年中供貨！蕭美琴透露：積極向美爭取額外釋出 - 政治 - 自由時報電子報
  </title>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0, minimum-scale=1.0" name="viewport"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="自由時報電子報" property="og:site_name"/>
  <meta content="自由時報電子報" name="author"/>
  <meta content="自由時報電子報" name="dcterms.rightsHolder"/>
  <meta content="自由時報電子報" name="copyright"/>
  <meta content="多管道確保疫苗年中供貨！蕭美琴透露：積極向美爭取額外釋出 - 政治 - 自由時報電子報" property="og:title"/>
  <meta content="蕭美琴,疫苗,美國,武漢肺炎,新型冠狀病毒,COV

In [118]:
#標題
t=ltn_news.find('title').text
t2=t.split('-')
title=t2[0]
#分類
senction=ltn_news.find('script',type="application/ld+json").contents[0].split('\n')[7].split(':')[1][2:-2]

In [119]:
title

'多管道確保疫苗年中供貨！蕭美琴透露：積極向美爭取額外釋出 '

In [16]:
senction

'國際'

In [17]:
print(title,senction)

對台伸援手！《路透》爆美國政府介入協調 加快疫苗交貨速度  國際


In [18]:
date=ltn_news.find('meta',property="article:published_time")['content'][0:10]
print(date)

2021-05-18


In [19]:
#ltn_news.find_all('meta')

In [120]:
def count_word(x):
    chinese=[]
    english=[]
    number=[]
    en=''
    num=''
    for i in list(x):

        if i in string.ascii_letters:
            en+=i
        elif i.isdigit():
            num+=i
        else:
            if en !='':
                english.append(en)
            if num!='':
                number.append(num)
            en=''
            num=''
            if i !=' ':
                chinese.append(i)
    words=len(english)+len(chinese)+len(number)
    return words

In [122]:
#內文
con_list=ltn_news.find_all('p')[3:]

content=''
for i  in con_list:
    try:
        if i.text[-1] in ['。','？','?','！'] and i.text!='「武漢肺炎專區」請點此，更多相關訊息，帶您第一手掌握。':
            content+=i.text
        else:
            pass
    except:
        pass
#字數
words=count_word(content)
#作者
a1=content.find('〔')
a2=content.find('／')
author=content[a1+1:a2]
if '記者' in author :
    author=author.replace('記者','')

In [123]:
words

475

In [100]:
content

''

In [24]:
#https://ec.ltn.com.tw/article/breakingnews/3382683

In [25]:
author

'即時新聞'

In [26]:
ltn_news.find_all('span',attrs={'class':'ph_b ph_d1'})

[]

In [155]:
n_p=len(ltn_news.find_all('div',attrs={'class':'photo boxTitle'}))+len(ltn_news.find_all('span',attrs={'class':'ph_b ph_d1'}))


In [156]:
n_p

1

In [29]:
#ltn_news.find_all('iframe',allow='accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture')

In [ ]:
"true" frameborder="0" height="40" scrolling="no"

In [69]:
ltn_news.find_all('iframe')[0]['allowtransparency']

'true'

In [176]:
v=ltn_news.find_all('iframe')
v2=[]
v3=[]
for i in v:
    try:
        if 'autoplay' in i['allow']:
            if 'facebook' in i['src']:
                v3.append(i)
            else:
                v2.append(i)
    except:
        pass
v_p=len(v2)

In [177]:
ltn_news.find_all('iframe')[1]['src']

'https://www.youtube.com/embed/GWC3HZgR8wA'

In [183]:
n_p

1

In [178]:
v_p

2

In [179]:
ltn_news.find_all('iframe')

[<iframe allowtransparency="true" frameborder="0" height="40" scrolling="no" src="https://www.facebook.com/plugins/like.php?href=https://news.ltn.com.tw/news/life/breakingnews/3103655" style="border:none;overflow:hidden;" width="40"></iframe>,
 <iframe allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen="" frameborder="0" height="315" src="https://www.youtube.com/embed/GWC3HZgR8wA" width="560"></iframe>,
 <iframe allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen="" frameborder="0" height="315" src="https://www.youtube.com/embed/IFX_1Y5MSUM" width="560"></iframe>,
 <iframe allowfullscreen="true" class="" frameborder="0" height="490px" id="f275c78e0f93108" name="f275c78e0f93108" scrolling="no" src="https://www.facebook.com/plugins/like_box.php?app_id=&amp;channel=https%3A%2F%2Fstaticxx.facebook.com%2Fconnect%2Fxd_arbiter.php%3Fversion%3D44%23cb%3Df1f2ec5b72ec584%26domain%3Dnews.ltn.com.tw%26origin%3

In [180]:
https://news.ltn.com.tw/news/world/breakingnews/3496920

SyntaxError: invalid syntax (<ipython-input-180-56df62f2c777>, line 1)

In [181]:
ltn_news.find_all('iframe')[1]['allow']

'accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture'

In [182]:
words==0

False